# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

#### EDA

In [2]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 4 columns):
id          26248 non-null int64
message     26248 non-null object
original    10184 non-null object
genre       26248 non-null object
dtypes: int64(1), object(3)
memory usage: 820.3+ KB


In [4]:
len(messages['id'].unique()) #does not match total nbumber, some are depulicates

26180

In [5]:
messages[messages['id'].duplicated(keep=False)].sort_values("id") #135 depulicates id

,id,message,original,genre
162,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct
163,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct
655,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct
656,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct
709,862,What is the address of the radio station? I as...,Ki adres radyo a? Paske m bezwen al depoze dos...,direct
710,862,What is the address of the radio station? I as...,Ki adres radyo a? Paske m bezwen al depoze dos...,direct
1407,1652,"please we need water, food and tents, we have ...","p jwen dlo, manje, tant pou nou demi nou gen 1...",direct
1408,1652,"please we need water, food and tents, we have ...","p jwen dlo, manje, tant pou nou demi nou gen 1...",direct
2131,2446,How much money did TV Latino American collect ...,Konbyen kob tv latino america ranmase miami,direct
2130,2446,How much money did TV Latino American collect ...,Konbyen kob tv latino america ranmase miami,direct


#### Clean

In [6]:
#remove duplicates rows who have he same id=df
messages=messages.drop_duplicates('id')

In [7]:
#make sure all rows with duplicated id has only keep one
assert(len(messages['id'].unique())==messages.shape[0])

In [8]:
#check message index
# #message=26180
#some duplicated messages has been removed thus some row index missing, 
#it could be the missing that produces extra nan rows when CONCAT MESSAGE and CATEGORIES
messages.index.values

array([    0,     1,     2, ..., 26245, 26246, 26247])

#### EDA

In [9]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [10]:
# #unique values < #rows thus duplicated exists
len(categories['id'].unique())

26180

In [11]:
#check duplicated id and keep one only
categories[categories['id'].duplicated(keep=False)].sort_values("id") #135 depulicates id

,id,categories
162,202,related-1;request-1;offer-0;aid_related-1;medi...
163,202,related-1;request-1;offer-0;aid_related-1;medi...
655,804,related-2;request-0;offer-0;aid_related-0;medi...
656,804,related-2;request-0;offer-0;aid_related-0;medi...
709,862,related-0;request-0;offer-0;aid_related-0;medi...
710,862,related-1;request-0;offer-0;aid_related-0;medi...
1407,1652,related-1;request-1;offer-0;aid_related-1;medi...
1408,1652,related-1;request-1;offer-0;aid_related-1;medi...
2131,2446,related-1;request-0;offer-0;aid_related-0;medi...
2130,2446,related-1;request-0;offer-0;aid_related-0;medi...


#### Clean

In [12]:
#check duplicated id and keep one only
categories=categories.drop_duplicates('id')


In [13]:
#make sure all rows with duplicated id has only keep one
assert(len(categories['id'].unique())==categories.shape[0])

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [14]:
# merge datasets
df = messages.merge(categories, how='outer',\
                               on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [15]:
df['id'][df.duplicated('id')]

Series([], Name: id, dtype: int64)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26180 entries, 0 to 26179
Data columns (total 5 columns):
id            26180 non-null int64
message       26180 non-null object
original      10153 non-null object
genre         26180 non-null object
categories    26180 non-null object
dtypes: int64(1), object(4)
memory usage: 1.2+ MB


In [17]:
assert(len(df['id'].unique())==df.shape[0])

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [18]:
# create a dataframe of the 36 individual category columns
categories = df['categories'].str.split(';',expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [19]:
# select the first row of the categories dataframe
row = categories.iloc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.values
print(category_colnames)

['related-1' 'request-0' 'offer-0' 'aid_related-0' 'medical_help-0'
 'medical_products-0' 'search_and_rescue-0' 'security-0' 'military-0'
 'child_alone-0' 'water-0' 'food-0' 'shelter-0' 'clothing-0' 'money-0'
 'missing_people-0' 'refugees-0' 'death-0' 'other_aid-0'
 'infrastructure_related-0' 'transport-0' 'buildings-0' 'electricity-0'
 'tools-0' 'hospitals-0' 'shops-0' 'aid_centers-0' 'other_infrastructure-0'
 'weather_related-0' 'floods-0' 'storm-0' 'fire-0' 'earthquake-0' 'cold-0'
 'other_weather-0' 'direct_report-0']


In [20]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [21]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str.get(-1) 
    # convert column from string to numeric
    categories[column] =  categories[column].astype('int32')
    
categories.head()

,1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#TODO: fix this
#work around: rename first columns label since it is modified with unknown reason
categories = categories.rename(columns={"1":"related-1"})

categories.head()

,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#show count of unique values of each column
categories.nunique()

related-1                   3
request-0                   2
offer-0                     2
aid_related-0               2
medical_help-0              2
medical_products-0          2
search_and_rescue-0         2
security-0                  2
military-0                  2
child_alone-0               1
water-0                     2
food-0                      2
shelter-0                   2
clothing-0                  2
money-0                     2
missing_people-0            2
refugees-0                  2
death-0                     2
other_aid-0                 2
infrastructure_related-0    2
transport-0                 2
buildings-0                 2
electricity-0               2
tools-0                     2
hospitals-0                 2
shops-0                     2
aid_centers-0               2
other_infrastructure-0      2
weather_related-0           2
floods-0                    2
storm-0                     2
fire-0                      2
earthquake-0                2
cold-0    

In [25]:
#categories[categories['related-1']==2]['related-1'].value_counts()

In [26]:
#replace class2 to Mode as we need all columns binary class but related-1 has 3 classes
categories['related-1'] = categories['related-1'].replace(2, categories['related-1'].mode()[0])

In [27]:
#make sure only 0 and 1
categories['related-1'].value_counts()

1    20064
0     6116
Name: related-1, dtype: int64

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [28]:
# drop the original categories column from `df`

df = df.drop('categories', axis=1)
df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [29]:
df.count()

id          26180
message     26180
original    10153
genre       26180
dtype: int64

In [30]:
#concatenate the original dataframe with the new `categories` dataframe
## CONFIRM THE REASON EXTRA ROWS WITH NAN OCCUR AFTER CONCAT IS DUE TO INDEXES MISMATCH BETWEEN 2 GIVEN DATAFRAME
#TODO: why concat results in int32 -> float64??
## issue fixed after reset_index...
df = pd.concat([df.reset_index(drop=True), categories.reset_index(drop=True)],axis=1)
df.head()

,id,message,original,genre,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26180 entries, 0 to 26179
Data columns (total 40 columns):
id                          26180 non-null int64
message                     26180 non-null object
original                    10153 non-null object
genre                       26180 non-null object
related-1                   26180 non-null int32
request-0                   26180 non-null int32
offer-0                     26180 non-null int32
aid_related-0               26180 non-null int32
medical_help-0              26180 non-null int32
medical_products-0          26180 non-null int32
search_and_rescue-0         26180 non-null int32
security-0                  26180 non-null int32
military-0                  26180 non-null int32
child_alone-0               26180 non-null int32
water-0                     26180 non-null int32
food-0                      26180 non-null int32
shelter-0                   26180 non-null int32
clothing-0                  26180 non-null int32
money-

In [32]:
df[df.duplicated('id')] #concat produces Nan id

,id,message,original,genre,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 6. Duplicated 

In [ ]:
#ALREADY FIXED DUPLICATED ISSUE ON PREVIOUS STEPS

In [33]:
# check number of duplicates
df.duplicated().values.sum()

0

In [34]:
# drop duplicates
df = df[~df.duplicated()]

In [35]:
# check number of duplicates
df.duplicated().values.sum()

0

In [36]:
##check datatype and null
##no need to remove null since RF can handles
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26180 entries, 0 to 26179
Data columns (total 40 columns):
id                          26180 non-null int64
message                     26180 non-null object
original                    10153 non-null object
genre                       26180 non-null object
related-1                   26180 non-null int32
request-0                   26180 non-null int32
offer-0                     26180 non-null int32
aid_related-0               26180 non-null int32
medical_help-0              26180 non-null int32
medical_products-0          26180 non-null int32
search_and_rescue-0         26180 non-null int32
security-0                  26180 non-null int32
military-0                  26180 non-null int32
child_alone-0               26180 non-null int32
water-0                     26180 non-null int32
food-0                      26180 non-null int32
shelter-0                   26180 non-null int32
clothing-0                  26180 non-null int32
money-

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [38]:
engine = create_engine('sqlite:///DisasterResponse.db')
df.to_sql('DisasterResponse', engine, if_exists='replace', index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.